In [6]:
import pandas as pd
import lyricsgenius

In [824]:
df = pd.read_csv('one_direction_songs.csv')

In [825]:
df['Song'] = df['Song'].apply(lambda x: x.strip('"'))

In [826]:
df['Artist(s)'] = df['Artist(s)'].apply(lambda x: x if x=='One Direction' else None)

In [827]:
df = df.dropna()

In [828]:
def split_every_two(writers):
    parts = writers.split()
    return [' '.join(parts[i:i+2]) for i in range(0, len(parts), 2)]
    
df['Writer(s)'] = df['Writer(s)'].apply(split_every_two)

In [829]:
df

,Song,Artist(s),Writer(s),Album(s),Year
0,18,One Direction,"[Ed Sheeran, Oliver Frank]",Four,2014
1,A.M.,One Direction,"[Niall Horan, Liam Payne, Harry Styles, Louis ...",Made in the A.M.,2015
2,Act My Age,One Direction,"[John Ryan, Julian Bunetta, Ed Drewett]",Four,2014
3,Alive,One Direction,"[Julian Bunetta, John Ryan, Jamie Scott, Louis...",Midnight Memories,2013
4,Another World,One Direction,"[Achraf Jannusi, Bilal Hajji, Eric Sanicola, G...",Up All Night,2011
...,...,...,...,...,...
89,What Makes You Beautiful,One Direction,"[Rami Yacoub, Carl Falk, Savan Kotecha]",Up All Night,2011
90,Where Do Broken Hearts Go,One Direction,"[Harry Styles, Julian Bunetta, Ruth-Anne Cunni...",Four,2014
91,Why Don't We Go There,One Direction,"[Steve Robson, Claude Kelly, Wayne Hector, Lou...",Midnight Memories,2013
93,Wolves,One Direction,"[Niall Horan, Liam Payne, Will Champlin, Andre...",Made in the A.M.,2015


In [ ]:
from lyricsgenius import Genius

genius = Genius('tfW8z_jNFSh-DTnS3qyHIU4zPXeWBSBX_ybqVJ9cgSai1OeXyImORVT0r7007F5w')
artist = genius.search_artist('One Direction', max_songs=100)

In [959]:
import re
for song in artist.songs:
    with open(f'songs_lyrics/{'_'.join(song.title.strip('?').split())}.txt', 'w', encoding='utf-8') as output:
        text = ' '.join(song.lyrics.split('Lyrics')[1:])
        text = text.split('Embed')[0]
        nums= re.findall(r'\d+', text)
        if nums:
            last = nums[-1]
            pos_last = text.rfind(nums[-1])
            text = text[:pos_last]
        output.write(song.title+'\n')
        output.write(text)

In [981]:
def get_lyrics(song):
    with open(f'songs_lyrics/{'_'.join(song.strip('?').split(' '))}.txt', 'r', encoding='utf-8') as file:
        return ' '.join(file.readlines()[1:])

In [982]:
df['Lyrics'] = df['Song'].apply(get_lyrics)

### Getting YouTube views

In [119]:
from selenium import webdriver
from selenium.webdriver.common.by import By 
import time

In [269]:
driver = webdriver.Firefox()

In [217]:
def click_button_cookies():
    try:
        btns = driver.find_elements(By.CSS_SELECTOR, '[class="yt-spec-button-shape-next yt-spec-button-shape-next--filled yt-spec-button-shape-next--mono yt-spec-button-shape-next--size-m"]')
        btns[0].click()
    except:
        pass

In [218]:
base_string = 'https://www.youtube.com/results?search_query=one+direction+'

In [219]:
def get_video_views(title):
    rem_part = '+'.join(title.lower().split(' '))
    driver.get(base_string+rem_part)
    time.sleep(2)
    click_button_cookies()
    time.sleep(2)
    meta = driver.find_element(By.CSS_SELECTOR, 'div #metadata-line span')
    return(' '.join(meta.text.split(' ')[:2]))

In [ ]:
# df['Views'] = df['Song'].apply(get_video_views)
df['Views'] = df['Views'].apply(lambda x: x.replace('Mln', '').strip())
df['Views'] = df['Views'].apply(lambda x: x.replace('Mrd', '.000').strip())

### Making up Lyrics

In [983]:
df['Lyrics'] = df['Lyrics'].apply(lambda x: x.replace('\n', ''))

In [984]:
df_views = pd.read_csv('df_views.csv')

In [985]:
df_views['Views'] = df_views['Views'].str.replace(',', '.').astype(float)

In [986]:
df.reset_index(drop=True, inplace=True)
df['Views'] = df_views['Views']

### Getting Words per singer

In [987]:
singers_names = ['Harry', 'Liam', 'Louis', 'Niall', 'Zayn', 'All']

In [991]:
import re
def append_singing_part(dict_singers, singer, lyrics):
    dict_singers[singer].append(lyrics)

def get_singers_words(text):
    dict_singers = {name: [] for name in singers_names}
    pattern = r"(?=\[)"
    sections = re.split(pattern, text)
    sections = [section for section in sections if section]
    in_pattern = r"\[([^:]+): ([^]]+)\] (.+)"
    for section in sections:
        match = re.match(in_pattern, section)
        if match:
            singer = match.group(2)
            lyrics = match.group(3)
            singer = singer.replace('&', ',')
            if 'All' in singer:
                append_singing_part(dict_singers, 'All', lyrics)   
            elif ',' in singer:
                singers = singer.split(', ')
                for sing in list(set(singers)):
                    append_singing_part(dict_singers, sing.strip(), lyrics)
            else:
                append_singing_part(dict_singers, singer, lyrics)
    return dict_singers
    

In [992]:
def clean_lyrics(text):
    pattern = r"\[[^]]*\]"
    clean_text = re.sub(pattern, "", text).strip()
    clean_text = re.sub(r"\s+", " ", clean_text)
    return clean_text

In [993]:
for singer in singers_names:
    df[singer] = df['Lyrics'].apply(lambda x: get_singers_words(x)[singer])
    df[singer] = df[singer].apply(lambda x: ' '.join(x))

In [1003]:
df['Lyrics'] = df['Lyrics'].apply(lambda x: clean_lyrics(x))

### Get singing percentages

In [996]:
def count_words(text):
    return len(text.split())

In [997]:
df['Lyrics_word_count'] = df['Lyrics'].apply(count_words)
df['All_word_count'] = df['All'].apply(count_words)

In [999]:
df['All%'] = df['All_word_count'] / df['Lyrics_word_count'] * 100
for singer in singers_names:
    df[singer+'_word_count'] = df[singer].apply(count_words)
    df[singer+'%'] = df[singer+'_word_count'] / df['Lyrics_word_count'] * 100

In [4]:
df = df.drop(singers_names, axis=1)

NameError: name 'df' is not defined

In [3]:
df.to_csv('df_main.csv', index=False)
df.to_excel('df_main.xlsx', index=False)

NameError: name 'df' is not defined